In [1]:
import pandas as pd

## Load Data

In [2]:
df = pd.read_csv('data/listings.csv.gz', index_col=0, compression='gzip', )

## Drop Irrelevant Features

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 95144 entries, 56229 to 1307795865634995863
Data columns (total 74 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   listing_url                                   95144 non-null  object 
 1   scrape_id                                     95144 non-null  int64  
 2   last_scraped                                  95144 non-null  object 
 3   source                                        95144 non-null  object 
 4   name                                          95144 non-null  object 
 5   description                                   92196 non-null  object 
 6   neighborhood_overview                         43463 non-null  object 
 7   picture_url                                   95134 non-null  object 
 8   host_id                                       95144 non-null  int64  
 9   host_url                                      95

Drop features which will certainly not contribute to a better machine learning model, as they mostly consist of scraping-data, textual-info, and urls.

In [4]:
feats = [
    'listing_url',
    'scrape_id',
    'last_scraped',
    'source',
    'description',
    'neighborhood_overview',
    'picture_url',
    'host_url',
    'host_name',
    'host_about',
    'host_thumbnail_url',
    'host_picture_url',
    'calendar_last_scraped'
    ]
df = df.drop(feats, axis=1)

Furthermore, remove features with more than 50% missing values.

In [5]:
print(f"Features with more than 50% missing values:\n{df.columns[df.isnull().mean() > .5]}")

df = df.dropna(thresh=.5*len(df), axis=1)

Features with more than 50% missing values:
Index(['host_neighbourhood', 'neighbourhood', 'neighbourhood_group_cleansed',
       'calendar_updated', 'license'],
      dtype='object')


Remove columns which hold only one unique value, making it redundant to include for training machine learning models.

In [6]:
print(f"Features with only one unique value:\n{df.columns[df.nunique() == 1]}")

df = df.loc[:,df.apply(pd.Series.nunique) != 1]

Features with only one unique value:
Index(['has_availability'], dtype='object')


In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 95144 entries, 56229 to 1307795865634995863
Data columns (total 55 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   name                                          95144 non-null  object 
 1   host_id                                       95144 non-null  int64  
 2   host_since                                    95137 non-null  object 
 3   host_location                                 72911 non-null  object 
 4   host_response_time                            62709 non-null  object 
 5   host_response_rate                            62709 non-null  object 
 6   host_acceptance_rate                          67888 non-null  object 
 7   host_is_superhost                             93396 non-null  object 
 8   host_listings_count                           95137 non-null  float64
 9   host_total_listings_count                     95

## Stardardizing and Formatting Data

Convert the price from object to float

In [8]:
df['price'] = df['price'].str.replace('$', '').str.replace(',', '').astype(float)

Convert percentages to decimal floats

In [9]:
percentage_cols = [
    'host_response_rate',
    'host_acceptance_rate'
]

for col in percentage_cols:
    df[col] = df[col].str.rstrip('%').astype(float) / 100

Convert f/t entries to booleans

In [10]:
boolean_cols = [
    'host_is_superhost',
    'host_has_profile_pic',
    'host_identity_verified',
    'instant_bookable'
]

m = {'t': True, 'f': False}

for col in boolean_cols:
    df[col] = df[col].map(m).astype(bool)

Convert date strings to DateTime objects

In [11]:
date_cols = [
    'host_since',
    'first_review',
    'last_review'
]

for col in date_cols:
    df[col] = pd.to_datetime(df[col])

In [12]:
df.info()
print("\n")
df.select_dtypes(include=['object']).info()

<class 'pandas.core.frame.DataFrame'>
Index: 95144 entries, 56229 to 1307795865634995863
Data columns (total 55 columns):
 #   Column                                        Non-Null Count  Dtype         
---  ------                                        --------------  -----         
 0   name                                          95144 non-null  object        
 1   host_id                                       95144 non-null  int64         
 2   host_since                                    95137 non-null  datetime64[ns]
 3   host_location                                 72911 non-null  object        
 4   host_response_time                            62709 non-null  object        
 5   host_response_rate                            62709 non-null  float64       
 6   host_acceptance_rate                          67888 non-null  float64       
 7   host_is_superhost                             95144 non-null  bool          
 8   host_listings_count                           95137 n

## Missing Data

In [13]:
df.isnull().sum()

name                                                0
host_id                                             0
host_since                                          7
host_location                                   22233
host_response_time                              32435
host_response_rate                              32435
host_acceptance_rate                            27256
host_is_superhost                                   0
host_listings_count                                 7
host_total_listings_count                           7
host_verifications                                  7
host_has_profile_pic                                0
host_identity_verified                              0
neighbourhood_cleansed                              0
latitude                                            0
longitude                                           0
property_type                                       0
room_type                                           0
accommodates                

Remove all rows of data where the price is unknown, as these cannot be used to train or test a model to predict the price.

In [14]:
print(f"Number of rows containing no price: {df['price'].isnull().sum()}")

df = df.dropna(subset=['price'])

Number of rows containing no price: 32367


Remove duplicate entries from the DataFrame

In [15]:
print(f"Number of duplicate entries: {df.duplicated().sum()}")

df = df.drop_duplicates()

Number of duplicate entries: 26


Remove rows which have more than half of the features missing.

In [16]:
print(f"Number of rows missing more than 50% of features: {df.isnull().mean(axis=1).gt(.5).sum()}")

df = df[df.isnull().mean(axis=1) < .5]

Number of rows missing more than 50% of features: 0


## Outliers

Remove all rows where the price is higher than the 99th percentile. This is to remove faulty listings, which can negatively impact the model.

In [19]:
print(f"Number of outlier above 99th percentile: {len(df[df['price'] > df['price'].quantile(0.99)])}")

df = df[df['price'] < df['price'].quantile(0.99)]

Number of outlier above 99th percentile: 626


---

In [20]:
df.to_csv('data/listings_cleaned.csv.gz', index=True, compression='gzip')